# torch installation
```
pip install torch torchvision --index-url=download.pytoch.org/whl/cu126
```

In [ ]:
# some imports
import torch
from ultralytics import YOLO, settings
from PIL import Image
from pathlib import Path

True

In [ ]:
# cuda settings
device= 'cuda' if torch.cuda.is_available() else 'cpu'

# ultralytics settings
settings.update({'datasets_dir': './datasets/CCPD'})
settings.update({'runs_dir': './runs'})
settings.update({'weights_dir': './weights'})

# CCPD settings
provinces = ["皖", "沪", "津", "渝", "冀", "晋", "蒙", "辽", "吉", "黑", "苏", "浙", "京", "闽", "赣", "鲁", "豫", "鄂", "湘", "粤", "桂", "琼", "川", "贵", "云", "藏", "陕", "甘", "青", "宁", "新", "警", "学", "O"]
alphabets = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'J', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W','X', 'Y', 'Z', 'O']
ads = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'J', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'O']
val_file_list='./datasets/CCPD/splits/val.txt'
datasets_root_dir=Path('./datasets/CCPD')

True

In [ ]:
# copy files into CCPD/ccpd_val\
# import shutil
# with open(datasets_root_dir / 'splits/val.txt', 'r') as f:
#     for line in f.readlines():
#         line = line.strip()
#         shutil.copyfile(datasets_root_dir / line, datasets_root_dir / 'ccpd_val/' / line.split('/')[-1])

In [ ]:
from pathlib import Path, WindowsPath
# CCPD dataloader
class CCPDLoader:
    def __init__(self, img_dir: str, img_size: int=640):
        self.img_files = list(Path(img_dir).glob("*.jpg"))
        self.img_size = img_size
        # CCPD annotation is it's filename
        self.annotations = [self.parse_filename(f) for f in self.img_files]
        
    def parse_filename(self, filename: WindowsPath):
        """
        Parse CCPD filename format:
        [Area]-[Tilt Angle]-[Bounding box coordinates]-[Four vertices locations]-[License plate number]-[Brightness]-[Blurriness].jpg
        Example: 025-95_113-154&383_386&473-386&473_177&454_154&383_363&402-0_0_22_27_27_33_16-37-15.jpg
        """
        parts = filename.stem.split('-')
        coords = list(map(int, parts[2].replace('&', ',').split(',')))
        points = np.array(coords, dtype=np.float32).reshape(4, 2)
        return {'points': points, 'plate_number': parts[-3]}
    
    def __len__(self):
        return len(self.img_files)
    
    def __getitem__(self, index):
        img = cv2.imread(str(self.img_files[index]))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        h, w = img.shape[:2]
        
        # resize with same aspect ratio
        r = min(self.img_size / h, self.img_size / w)
        new_h, new_w = int(h * r), int(w * r)
        img = cv2.resize(img, (new_w, new_h))
        img = np.ascontiguousarray(img)
        return img, self.annotations[index]

In [46]:
# filename=list(Path('./datasets/CCPD/ccpd_base').glob('*.jpg'))[0]
# parts=filename.stem.split('-')
# parts[-3]
# filename="025-95_113-154&383_386&473-386&473_177&454_154&383_363&402-0_0_22_27_27_33_16-37-15.jpg"
# filename.split('-')
with open(val_file_list,'r') as f:
    filename=f.readlines()
print(filename[1])




ccpd_base/0384159482759-87_93-367&427_654&550-656&527_369&554_379&441_666&414-0_0_13_19_33_27_24-95-81.jpg

